<a href="https://colab.research.google.com/github/Niltonguerra/Materia-de-processamento-de-linguagem-natural/blob/main/aula_do_dia_22_11_24_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import re

In [ ]:
original = 'olá!! Este é um exemplo de texto, com várias PONTUAÇÕES, símbolos #especiais, e Letras maiúsculas.'

texto_limpo = re.sub(r'[^A-Za-zÀ-ÿ\s]','',original)

texto_normalizado = texto_limpo.lower()


print(f'Texto original: {original}')
print(f'\nTexto limpo: {texto_limpo}')
print(f'\nTexto normalizado: {texto_normalizado}')

Texto original: olá!! Este é um exemplo de texto, com várias PONTUAÇÕES, símbolos #especiais, e Letras maiúsculas.

Texto limpo: olá Este é um exemplo de texto com várias PONTUAÇÕES símbolos especiais e Letras maiúsculas

Texto normalizado: olá este é um exemplo de texto com várias pontuações símbolos especiais e letras maiúsculas


In [ ]:
import nltk
from nltk.tokenize import word_tokenize

# Baixa o recurso de tokenização, se necessário
try:
    nltk.data.find('tokenizers/punkt')
except LookupError:
    nltk.download('punkt')

# Função para tokenizar texto com tratamento de erro
def tokenize_text(text):
    if not isinstance(text, str):
        raise ValueError("O texto de entrada deve ser uma string.")
    return word_tokenize(text)

# Exemplo de uso
texto_normalizado = "Aqui está o texto para ser tokenizado."
tokens = tokenize_text(texto_normalizado)

print(tokens)


['Aqui', 'está', 'o', 'texto', 'para', 'ser', 'tokenizado', '.']


In [ ]:
#
from nltk.corpus import stopwords

nltk.download('stopwords')

stopwords_pt = set(stopwords.words('portuguese'))
token_sem_stopwords = [palavra for palavra in tokens if palavra.lower() not in stopwords_pt]
print(token_sem_stopwords)

['Aqui', 'texto', 'tokenizado', '.']


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
#mimetização ou lemetização

from nltk.stem import RSLPStemmer

nltk.download('rslp')

stemmer = RSLPStemmer()
stemming = [stemmer.stem(palavra) for palavra in token_sem_stopwords]
print(stemming)

['aqu', 'text', 'token', '.']


[nltk_data] Downloading package rslp to /root/nltk_data...
[nltk_data]   Package rslp is already up-to-date!


In [12]:
# Passo 1 - Criar o corpus
# O corpus é uma lista de tuplas, onde cada tupla contém uma frase e um rótulo de classe ('positivo' ou 'negativo').
corpus = [
    ("Eu amo PLN", "positivo"),         # Exemplo de frase positiva
    ("Eu odeio bugs", "negativos"),     # Exemplo de frase negativa (observe o erro no rótulo "negativos" -> "negativo")
    ("Amo resolver problemas", "positivo"), # Exemplo de frase positiva
    ("Odeio erros", "negativo"),         # Exemplo de frase negativa
    ("Bugs legais","positivo"),
    ("Resolver","positivo"),
    ("Eu não","negativo")
    ]

# Passo 2 - Processar o texto
import re
from collections import defaultdict, Counter

# Função para pré-processar o texto
# Converte o texto para minúsculas e divide em palavras (tokens)
def preprocess_text(text):
    return re.findall(r'\b\w+\b', text.lower())  # Retorna uma lista de palavras encontradas no texto

# Processa o corpus aplicando o pré-processamento em cada frase
# Retorna uma lista de tuplas: (lista de palavras, rótulo)
processed_corpus = [(preprocess_text(text), label) for text, label in corpus]
print(processed_corpus)

# Passo 3 - Calculando probabilidades
# Inicializa contadores para as classes, palavras e total de palavras por classe
class_counts = Counter()            # Conta a quantidade de exemplos por classe
word_counts = defaultdict(Counter)  # Conta a ocorrência de palavras em cada classe
total_words = defaultdict(int)      # Total de palavras em cada classe

# Itera pelo corpus processado para calcular as contagens necessárias
for words, label in processed_corpus:
    class_counts[label] += 1  # Incrementa o contador da classe
    for word in words:
        word_counts[label][word] += 1  # Incrementa o contador da palavra para a classe correspondente
        total_words[label] += 1        # Incrementa o total de palavras na classe

# Calcula a probabilidade a priori de cada classe: P(classe)
# Isso é feito dividindo o número de exemplos em cada classe pelo total de exemplos
total_examples = sum(class_counts.values())
prior_probabilities = {cls: count / total_examples for cls, count in class_counts.items()}

# Função para calcular a probabilidade condicional de uma palavra dada uma classe: P(palavra|classe)
# Usa suavização de Laplace (alpha) para evitar problemas com palavras desconhecidas
def conditional_probability(word, label, alpha=1):
    return (word_counts[label][word] + alpha) / (total_words[label] + alpha * len(word_counts[label]))

# Passo 4 - Classificar um novo texto
# Função para prever a classe de um texto
def predict(text):
    words = preprocess_text(text)  # Pré-processa o texto de entrada
    probabilities = {}             # Armazena as probabilidades para cada classe

    # Calcula a probabilidade para cada classe
    for label in class_counts.keys():
        probabilities[label] = prior_probabilities[label]  # Começa com a probabilidade a priori: P(classe)
        for word in words:
            probabilities[label] *= conditional_probability(word, label)  # Multiplica pela probabilidade condicional: P(palavra|classe)

    # Retorna a classe com maior probabilidade e as probabilidades calculadas
    return max(probabilities, key=probabilities.get), probabilities

# Passo 5 - Teste com um novo texto
novo_texto = "eu amo resolver bugs"  # Texto a ser classificado
classe, probs = predict(novo_texto) # Chama a função de predição

# Imprime o resultado
print(f'Texto: "{novo_texto}"')           # Mostra o texto de entrada
print(f'Classe prevista: `{classe}`')     # Mostra a classe prevista
print(f'Probabilidades:')                 # Mostra as probabilidades calculadas
for label, prob in probs.items():
    print(f"{label}: {prob}")             # Mostra a probabilidade de cada classe


[(['eu', 'amo', 'pln'], 'positivo'), (['eu', 'odeio', 'bugs'], 'negativos'), (['amo', 'resolver', 'problemas'], 'positivo'), (['odeio', 'erros'], 'negativo'), (['bugs', 'legais'], 'positivo'), (['resolver'], 'positivo'), (['eu', 'não'], 'negativo')]
Texto: "eu amo resolver bugs"
Classe prevista: `negativos`
Probabilidades:
positivo: 0.00031389508928571425
negativos: 0.0004409171075837742
negativo: 0.00013950892857142856


In [31]:
# Passo 1 importação das bibliotecas a serem utilizadas
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

# Passo 2 dados exemplo
corpus = [
    "Eu amo PLN", "Eu odeio bugs", "Eu amo resolver problemas",
    "Odeio erros","Amo programação","Não gosto de falhas"
]

classes = ["negativo","negativo","positivo","negativo","positivo","negativo"]

# Passo 3 Pre processamento e vetorização
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(corpus)
y = classes

#Passo 4 Dividir os dados e treinar o modelo
X_train,X_test,y_train, y_test = train_test_split(X,y, test_size=0.3,random_state=42)

svm_model = SVC(kernel='linear')
svm_model.fit(X_train, y_train)


#Passo 5 Avaliar o modelo
y_pred = svm_model.predict(X_test)
print(classification_report(y_test,y_pred))


              precision    recall  f1-score   support

    negativo       1.00      0.50      0.67         2
    positivo       0.00      0.00      0.00         0

    accuracy                           0.50         2
   macro avg       0.50      0.25      0.33         2
weighted avg       1.00      0.50      0.67         2



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


análise desse relatório:

               precision    recall  f1-score   support

     negativo       1.00      0.50      0.67         2
     positivo       0.00      0.00      0.00         0

     accuracy                           0.50         2
     macro avg       0.50      0.25      0.33         2
     weighted avg       1.00      0.50      0.67         2

#####Métricas por Classe:
Cada classe (aqui negativo e positivo) tem métricas calculadas individualmente.

#####Métricas Agregadas:

Métricas como accuracy, macro avg, e weighted avg mostram uma visão geral do modelo.
Termos Importantes
#####Precision (Precisão):
A proporção de previsões corretas para uma classe específica, ou seja, True Positives / (True Positives + False Positives).
Mede o quão preciso o modelo é ao prever essa classe.
#####Recall (Sensibilidade):
A proporção de exemplos verdadeiros de uma classe que o modelo conseguiu identificar, ou seja, True Positives / (True Positives + False Negatives).
Mede o quão bem o modelo encontra todos os exemplos de uma classe.
F1-Score:
A média harmônica entre precisão e recall. É útil quando há um desequilíbrio entre as classes.
Fórmula: 2 * (precision * recall) / (precision + recall).
#####Support (Suporte):
O número total de exemplos reais de cada classe no conjunto de teste.
####Análise do Relatório:
#####Classe: negativo
Precisão (Precision): 1.00
Todas as previsões feitas para a classe negativo estavam corretas (nenhum falso positivo).
#####Recall (Sensibilidade): 0.50
Apenas 50% dos exemplos reais da classe negativo foram corretamente identificados.
F1-Score: 0.67
Combinação de precisão e recall; mostra um desempenho moderado nesta classe.
Support: 2
Existiam 2 exemplos da classe negativo no conjunto de teste.
#####Classe: positivo
Precisão (Precision): 0.00
Nenhuma previsão correta foi feita para a classe positivo.
#####Recall (Sensibilidade): 0.00
O modelo não identificou nenhum exemplo da classe positivo.
F1-Score: 0.00
O desempenho nesta classe foi nulo.


In [33]:
# 1. Importar Bibliotecas
# Importa bibliotecas essenciais para pré-processamento, modelagem e avaliação.
import nltk  # Para manipular dados textuais e datasets embutidos
from sklearn.feature_extraction.text import TfidfVectorizer  # Para transformar textos em vetores TF-IDF
from sklearn.model_selection import train_test_split  # Para dividir o conjunto de dados em treino e teste
from sklearn.naive_bayes import MultinomialNB  # Modelo Naive Bayes para classificação
from sklearn.svm import SVC  # Máquina de Vetores de Suporte (SVM) para classificação
from sklearn.metrics import classification_report  # Para avaliar o desempenho do modelo
from sklearn.preprocessing import LabelEncoder  # Para transformar rótulos categóricos em numéricos

# Baixar o dataset de exemplo
nltk.download('movie_reviews')  # Faz o download do dataset "movie_reviews" do NLTK
from nltk.corpus import movie_reviews  # Importa o corpus de análises de filmes

# 2. Preparação dos dados

# Coleta de textos e classes
# O dataset contém textos (palavras de reviews de filmes) e suas categorias ('pos' ou 'neg').
# Aqui, combinamos as palavras de cada documento em uma string e associamos com sua categoria.
documents = [
    (" ".join(movie_reviews.words(fileid)), category)  # Junta as palavras de cada documento em uma única string
    for category in movie_reviews.categories()  # Itera pelas categorias ('pos' e 'neg')
    for fileid in movie_reviews.fileids(category)  # Itera pelos arquivos de cada categoria
]

# Separar textos e rótulos
# Separa o texto das análises (texts) e suas respectivas categorias (labels).
texts, labels = zip(*documents)

# Transformar rótulos (positivo/negativo) em 0 e 1
# Usa o LabelEncoder para converter as categorias ('pos' e 'neg') em números (0 e 1).
label_encoder = LabelEncoder()
labels = label_encoder.fit_transform(labels)

# Dividir dados em treino e teste
# Divide o dataset em 70% para treino e 30% para teste.
texts_train, texts_test, labels_train, labels_test = train_test_split(
    texts, labels, test_size=0.3, random_state=42  # Define uma semente para garantir reprodutibilidade
)

# 3. Representação do texto com TF-IDF

# Criar o vetorizador TF-IDF
# O TF-IDF converte os textos em vetores numéricos, representando a relevância das palavras nos documentos.
# Limitamos a 5000 palavras mais frequentes para reduzir a dimensionalidade.
vectorizer = TfidfVectorizer(max_features=5000)

# Ajustar e transformar os textos de treino
# O método fit_transform ajusta o vetor TF-IDF aos dados de treino e os transforma em vetores numéricos.
X_train = vectorizer.fit_transform(texts_train)

# Transformar os textos de teste
# Apenas transforma os textos de teste com base no modelo ajustado aos dados de treino.
X_test = vectorizer.transform(texts_test)

# 4. Treinar os modelos

# Treinamento do Naive Bayes
# Ajusta o modelo Naive Bayes Multinomial aos vetores de treino e seus rótulos.
nb_model = MultinomialNB()
nb_model.fit(X_train, labels_train)

# Predição do Naive Bayes
# Gera as previsões do modelo Naive Bayes para os textos de teste.
nb_predictions = nb_model.predict(X_test)

# Treinamento do SVM
# Ajusta o modelo SVM com kernel linear aos vetores de treino e seus rótulos.
svm_model = SVC(kernel="linear", random_state=42)
svm_model.fit(X_train, labels_train)

# Predição do SVM
# Gera as previsões do modelo SVM para os textos de teste.
svm_predictions = svm_model.predict(X_test)

# 5. Avaliação

# Avaliação do Naive Bayes
# Exibe o relatório de classificação para o modelo Naive Bayes.
# Inclui métricas como precisão (precision), sensibilidade (recall) e F1-Score para cada classe.
print("Naive Bayes Performance:")
print(classification_report(labels_test, nb_predictions, target_names=label_encoder.classes_))

# Avaliação do SVM
# Exibe o relatório de classificação para o modelo SVM.
print("SVM Performance:")
print(classification_report(labels_test, svm_predictions, target_names=label_encoder.classes_))


[nltk_data] Downloading package movie_reviews to /root/nltk_data...
[nltk_data]   Package movie_reviews is already up-to-date!


Naive Bayes Performance:
              precision    recall  f1-score   support

         neg       0.79      0.84      0.81       302
         pos       0.82      0.77      0.80       298

    accuracy                           0.80       600
   macro avg       0.80      0.80      0.80       600
weighted avg       0.80      0.80      0.80       600

SVM Performance:
              precision    recall  f1-score   support

         neg       0.82      0.80      0.81       302
         pos       0.81      0.82      0.81       298

    accuracy                           0.81       600
   macro avg       0.81      0.81      0.81       600
weighted avg       0.81      0.81      0.81       600

